# CAO Points Analysis
***

In [1]:
# import required libraries
import requests as rq             # Convenient HTTP requests.
import re                         # Regular expressions.
import datetime as dt   
import pandas as pd
import numpy as np

# Get the current date and time in desired string format
nowstr = dt.datetime.now().strftime('%Y%m%d_%H%M%S')

<br>

**Error on server (reference function below)** <br>
The Server indicated one should decode as per: Content-Type: text/html; charset=iso-8859-1. However, one line of the html uses<br> 
\x96 which isn't defined in iso-8859-1. Therefore the similar decoding standard cp1252 will be  used. This standard is very similar but<br>
includes \x96.

In [2]:

def website_data_handler(url, regex):
    
    response = rq.get(url)                                  # Make a get request to 2021 CAO points URL
    response.encoding = 'cp1252'                            # Specify encoding method as cp1252 despir what server recommended
    
    no_lines = 0                                            # Keep track of courses counted in loop
    
    with open('data/cao2021.csv', 'w') as f:                    # Open the csv file for writing         
        for line in response.iter_lines():                          # Loop through lines of the response 
            dline = line.decode('cp1252')                           # decode the line 
            if regex.fullmatch(dline):                          # Match only the lines representing courses.       
                no_lines = no_lines + 1                                 # Add one to the lines counter if match occured      
                dline_adj = re.sub(" ", "  ", dline, count = 1)         # Substitute first space character with double space     
                linesplit = re.split('   +', dline_adj)                 # Split the line on three or more spaces.  
                f.write(','.join(linesplit) + '\n')                     # Rejoin the substrings with commas in between.         
    print(f"Total number of lines is {no_lines}.")         # Print the total number of processed lines
    

In [3]:
url_2021 = 'http://www2.cao.ie/points/l8.php'                                                       # Make a get request to 2021 CAO points URL
regex_course_2021 = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)(#?)(?:([AQ]{3})|([V]{1})|([0-9]{3}))(\*?) *')     # define regular expression
website_data_handler(url_2021, regex_course_2021)                                                       # pass paremeters into parse & write function


# Regular Expression breakdown:                Explanation for what characters will be parsed:    
#**************************************        ***************************************************************************************************
# ([A-Z]{2}[0-9]{3})                           first 2 chars (A-Z), next 3 chars, 0-9 followed by two spaces
# (.*)                                         Matches any single character except newline zero or more times
# (?:([AQ]{3})|([V]{1})|([0-9]{3}))            Match (3 x [AQ]) or (1 x V) or (3 x [0-9]) zero or one time
# (\*?) *'                                    \(escaped) "*" so dont treat it as quantifier, treat it by itself. Zero or one * character "?" 
# space *                                     any number of spaces



Total number of lines is 923.


In [4]:
#new_col_order = ['Year', 'CATEGORY (i.e.ISCED description)', 'HEI', 'COURSE TITLE',              
                 #'COURSE CODE2','R1 POINTS', 'R2 POINTS', 'EOS Mid-point', 'EOS', 
                 #'R1 Random *',  'R2 Random*', 'EOS Random *', 'Test/Interview #'] 


def data_formatter_2021():
    
    # read csv from folder and pass list of column headers
    read_path = 'data/cao2021.csv'
    header_list_2021 = ['COURSE CODE', 'COURSE TITLE', 'R1 POINTS', 'R2 POINTS']       
    df = pd.read_csv(read_path, names = header_list_2021)                                                                                                                                               
    
    df.insert(0, 'YEAR', 2021) 
    
    # insert 3 new conditional columns via string operations 
    # explanation : (if col contains string x, return x, else return y)
    df['R1 Random*'] = np.where(df['R1 POINTS'].str.contains('\*'), '*', '')              
    df['R2 Random*'] = np.where(df['R2 POINTS'].str.contains('NaN'), '',               
                         np.where(df['R2 POINTS'].str.contains('\*'), '*', ''))
    
    df['Test/Interview #'] = np.where((df['R1 POINTS'].str.contains('#')) | 
                                      (df['R2 POINTS'].str.contains('#')), '#', '')  
    
    #df_cols = df.columns.tolist()
    
    cols = ['a', 'b', 'c']
    num = [0, 0, 0]
    df.assign(**dict(zip(cols, num)))
    

    
    # remove '*' and '#' from points columns  
    df['R1 POINTS'].replace(regex = ['\*', '#'], value = '', inplace = True)            
    df['R2 POINTS'].replace(regex = ['\*', '#'], value = '', inplace = True)
    return df
    
    
df_2021 = data_formatter_2021()
df_2021.head(50)

,YEAR,COURSE CODE,COURSE TITLE,R1 POINTS,R2 POINTS,R1 Random*,R2 Random*,Test/Interview #
0,2021,AL801,Software Design for Virtual Reality and Gaming,300,NaN,,,
1,2021,AL802,Software Design in Artificial Intelligence for...,313,NaN,,,
2,2021,AL803,Software Design for Mobile Apps and Connected ...,350,NaN,,,
3,2021,AL805,Computer Engineering for Network Infrastructure,321,NaN,,,
4,2021,AL810,Quantity Surveying,328,NaN,,,
5,2021,AL820,Mechanical and Polymer Engineering,327,NaN,,,
6,2021,AL830,General Nursing,451,444,*,,
7,2021,AL832,Mental Health Nursing,440,431,*,,
8,2021,AL835,Pharmacology,356,NaN,,,
9,2021,AL836,Nutrition and Health Science,346,NaN,,,


# 2020 data

In [5]:
def data_handler_2020():
    
    # read in excel 2020 data 
    read_url = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'
    
    
    df = pd.read_excel(read_url, skiprows = 10)     
    #df = pd.read_excel('data/CAOPointsCharts2020.xlsx', skiprows = 10)  

    
    df.insert(0, 'YEAR', 2020) 
    df = df.loc[df.loc[:, 'LEVEL'] == 8]                                                   # re-define df with level 8 courses filtered   
    df.drop(['avp', 'v', 'LEVEL'], axis = 1, inplace = True)                               # drop 'avp' and 'v' columns as these are not in 2019/2021 data
    df = df.iloc[:,:-8]                                                                    # drop last 8 empty cols dropped 
       
    
    # define col order list
    new_col_order = ['YEAR', 'CATEGORY (i.e.ISCED description)', 'HEI', 'COURSE TITLE',              
                     'COURSE CODE2','R1 POINTS', 'R2 POINTS', 'EOS Mid-point', 'EOS', 
                     'R1 Random *',  'R2 Random*', 'EOS Random *', 'Test/Interview #']  
    # re-order columns
    df = df.reindex(columns = new_col_order)                                                
                       
    # re-name columns from dict key to dict value 
    df.rename(columns = {'CATEGORY (i.e.ISCED description)': 'CATEGORY', 'Hei': 'HEI', 
                        'COURSE CODE2': 'COURSE CODE', 'EOS': 'FINAL SEASON POINTS',
                         'EOS Mid-point': 'MID SEASON POINTS', 'EOS Random *': 
                         'FINAL SEASON POINTS RANDOM*'}, inplace = True)      

    
    col_str_replace = ['R1 POINTS', 'R2 POINTS', 'MID SEASON POINTS', 'FINAL SEASON POINTS']              # define list cols for string replace operation 
    df[col_str_replace] = np.where(df[col_str_replace] == '#+matric', 'NaN', df[col_str_replace])         # replace '#+matric' with 'NaN'      
   
    return df
    


data_2020 = data_handler_2020()
data_2020

,YEAR,CATEGORY,HEI,COURSE TITLE,COURSE CODE,R1 POINTS,R2 POINTS,MID SEASON POINTS,FINAL SEASON POINTS,R1 Random *,R2 Random*,FINAL SEASON POINTS RANDOM*,Test/Interview #
0,2020,Business and administration,American College,International Business,AC120,209,NaN,280,209,NaN,NaN,NaN,NaN
1,2020,Humanities (except languages),American College,Liberal Arts,AC137,252,NaN,270,252,NaN,NaN,NaN,NaN
2,2020,Arts,National College of Art and Design,"First Year Art & Design (Common Entry,portfolio)",AD101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#
3,2020,Arts,National College of Art and Design,Graphic Design and Moving Image Design (portfo...,AD102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#
4,2020,Arts,National College of Art and Design,Textile & Surface Design and Jewellery & Objec...,AD103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2020,Arts,Waterford Institute of Technology,Arts (options),WD200,AQA,AQA,336,AQA,NaN,NaN,NaN,NaN
1460,2020,Information and Communication Technologies (ICTs),Waterford Institute of Technology,Software Systems Development,WD210,279,NaN,337,279,NaN,NaN,NaN,NaN
1461,2020,Information and Communication Technologies (ICTs),Waterford Institute of Technology,Creative Computing,WD211,271,NaN,318,271,NaN,NaN,NaN,NaN
1462,2020,Personal services,Waterford Institute of Technology,Recreation and Sport Management,WD212,270,NaN,349,270,NaN,NaN,NaN,NaN


In [13]:
# request url for raw pdf

url = 'http://www2.cao.ie/points/lvl8_19.pdf'
resp = rq.get(url)


# save raw pdf to disc in binary mode (i.e: makes no changes to file)
raw_pdf_2019 = 'data/cao_2019_raw_data.pdf'
with open(raw_pdf_2019, 'wb') as f:  
    f.write(resp.content)
    

In [72]:
# extract pdf into text and write to intermediate file
import pdfplumber

intermediate_path = 'data/cao_2019_intermediate.txt'
with open(intermediate_path, 'w') as f:    
    
    data_final = ''
    with pdfplumber.open(raw_pdf_2019) as pdf:
        for page in pdf.pages:
            data = page.extract_text()
            data_final += data        
    f.write(data_final)


# Compile the regular expression for matching lines.
regex = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')  

with open(intermediate_path, 'r') as file:    
    for line in file:      
        for match in re.finditer(regex, line):
            print(match.group())


    
###regex_course_2021 = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)(#?)(?:([AQ]{3})|([V]{1})|([0-9]{3}))(\*?) *')     # define regular expression
# Regular Expression breakdown:                Explanation for what characters will be parsed:    
#**************************************        ***************************************************************************************************
# ([A-Z]{2}[0-9]{3})                           first 2 chars (A-Z), next 3 chars, 0-9 followed by two spaces
# (.*)                                         Matches any single character except newline zero or more times
# (?:([AQ]{3})|([V]{1})|([0-9]{3}))            Match (3 x [AQ]) or (1 x V) or (3 x [0-9]) zero or one time
# (\*?) *'                                    \(escaped) "*" so dont treat it as quantifier, treat it by itself. Zero or one * character "?" 
# space *                                     any number of spaces

      

AL801 Software Design with Virtual Reality and Gaming 304 328
AL802 Software Design with Cloud Computing 301 306
AL803 Software Design with Mobile Apps and Connected Devices 309 337
AL805 Network Management and Cloud Infrastructure 329 442
AL810 Quantity Surveying 307 349
AL820 Mechanical and Polymer Engineering 300 358
AL830 General Nursing 410 429
AL832 Psychiatric Nursing 387 403
AL836 Nutrition and Health Science 352 383
AL837 Sports Science with Exercise Physiology 351 392
AL838 Biotechnology 302 372
AL839 Microbiology 322 371
AL840 Pharmaceutical Sciences 328 411
AL841 Athletic and Rehabilitation Therapy 423 430
AL842 Bioveterinary Science 310 366
AL843 Physical Activity and Health Science 310 330
AL850 Business 307 356
AL851 Business and Law 301 341
AL852 Accounting (with optional placement) 290 346
AL853 Accounting and Law 305 320
AL855 Hospitality Management (with International placement) 312 357
AL857 Digital Marketing 300 360
AL860 Social Care Practice 309 338
AL861 Animatio

In [10]:
'''
# The file path for the csv file.
csv_2019 = 'data/cao_2021.csv'

# open a csv for writing
with open(csv_2019, 'w') as f:
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')           # Write a header row.
    for line in resp.iter_lines():                                                # Loop through lines of the response.
        dline = line.decode('cp1252')
        print(dline)
'''

"\n# The file path for the csv file.\ncsv_2019 = 'data/cao_2021.csv'\n\n# open a csv for writing\nwith open(csv_2019, 'w') as f:\n    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')           # Write a header row.\n    for line in resp.iter_lines():                                                # Loop through lines of the response.\n        dline = line.decode('cp1252')\n        print(dline)\n"

In [11]:
'''
# read raw pdf from disc
import PyPDF2  

file = open('data/cao_2019_raw_data.pdf', 'rb')
reader = PyPDF2.PdfFileReader(file)
page_count = reader.numPages
#page1 = reader.getPage(0)
#pdf_data = page1.extractText()

data_all = ''
for page in range(page_count):
    text = reader.getPage(page)  
    data = text.extractText()    
    data_all += data
''';

In [28]:
'''
import pandas as pd
import numpy as np

def data_handler_2019(read_path, drop_row_num, drop_header_rows, header_list):  
    
    # read csv (note: gives error when i pass in read_path parameter....check this)
    df = pd.read_csv('data/cao_2019_raw_data.txt', skiprows = skip_row_num, sep = '\n')      
 
    df = df.iloc[:-1 , :]                                             # drop last row (page number)   
    header_rows = []                                                  # get list of header rows for future use
    drop_row_list = ['college', 'institute', 'university', 'school']  # make list of strings, will be used to remove header rows
    
    for row in df.iloc[:,0]:
        for item in drop_row_list:
            if item in row:
                header_rows.append(row)
                print(item)   
       
    df = df[~df.iloc[:, 0].isin(drop_header_rows)]                     # drop rows in list    
    array = df.values.reshape(41,4)                                    # reshape df to numpy array
    df_final = pd.DataFrame(array, columns = header_list)              # convert np.array to df and add  add headers
    df_final.to_csv(write_path)                                        # write to csv
    
    return df_final
    
   
# 2019 pdf variables
read_path = 'data/cao_2019_raw_data.txt'
skip_row_num = 19
header_list_2019 = ['COURSE CODE', 'COURSE TITLE', 'R1 POINTS', 'R2 POINTS']  
write_path = 'data/cao2019.csv' 

data_2019 = data_handler_2019(drop_header_rows, skip_row_num, drop_header_rows, header_list_2019)
data_2019.head(50)
''';

In [25]:
path = 'data/cao_2019_raw_data.txt'
   # read csv
df

NameError: name 'df' is not defined